In [43]:
import pandas as pd
import numpy as np
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import warnings
warnings.filterwarnings("ignore")

In [44]:
df = pd.read_csv('./Seoul.csv')
df

,Unnamed: 0,상세영업상태코드,도로명주소,사업장명,업태구분명,좌표정보(X),좌표정보(Y)
0,1,1,서울특별시 종로구 삼일대로 382 (관철동),상해,분식,198867.702591,451824.281126
1,2,1,서울특별시 관악구 신림로59길 15-13 4층 (신림동),미쿠,일식,193687.348199,442284.164025
2,3,1,서울특별시 구로구 구로동로22길 21 1층 (구로동),여름엔빙수겨울엔떡국,한식,189814.305919,443046.239859
3,4,1,서울특별시 강남구 삼성로85길 33 1층 102-2호 (대치동),카페디퍼,기타,204776.548264,444677.642985
4,5,1,서울특별시 동대문구 난계로30길 28-7 1층 (신설동),메이크데이(MakeDay),기타,202114.228397,452392.093759
...,...,...,...,...,...,...,...
121757,474932,1,서울특별시 동작구 대림로 55 1층 (신대방동),우동짜장&야식포차,횟집,191735.419996,442882.907854
121758,474933,1,서울특별시 중구 다산로 137 (신당동 1층),금성회관 약수역직영점,한식,200914.227438,450426.403396
121759,474934,1,서울특별시 광진구 동일로24길 28 1층 (화양동),블랙아웃(Black out),정종/대포집/소주방,205812.137123,449038.140512
121760,474937,1,서울특별시 강동구 상암로 50 (천호동),맛닭꼬 암사점,한식,211313.932580,449686.721186


In [45]:
search_word = df.사업장명
search_word

0                      상해
1                      미쿠
2              여름엔빙수겨울엔떡국
3                    카페디퍼
4          메이크데이(MakeDay)
               ...       
121757          우동짜장&야식포차
121758        금성회관 약수역직영점
121759    블랙아웃(Black out)
121760            맛닭꼬 암사점
121761               제주도집
Name: 사업장명, Length: 121762, dtype: object

In [47]:
options = webdriver.ChromeOptions()
options.add_argument('headless')

path = 'C:/Users/USER/TIL/chromedriver'

source_url = "https://map.kakao.com/"

page_urls = []

driver = webdriver.Chrome(executable_path=path, chrome_options=options)

for name in search_word:
    try:
        driver.get(source_url)

        search_bar = driver.find_element_by_xpath("//input[@id='search.keyword.query']")
        search_bar.send_keys(name)

        searchbutton = driver.find_element_by_xpath("//button[@id='search.keyword.submit']")
        driver.execute_script("arguments[0].click();", searchbutton)

        WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '//*[@id="info.search.place.list"]/li[2]/div[3]/strong/a[1]')))

        html = driver.page_source

        soup = BeautifulSoup(html, "html.parser")
        moreview = soup.find(name="a", attrs={"class":"moreview"})
    
        page_url = moreview.get("href")
        print(page_url)
        page_urls.append(page_url)
    except:
        pass

driver.close()

https://place.map.kakao.com/2081285517
https://place.map.kakao.com/1347932853
https://place.map.kakao.com/637651089
https://place.map.kakao.com/1536862526
https://place.map.kakao.com/422769336
https://place.map.kakao.com/202750660
https://place.map.kakao.com/16326716
https://place.map.kakao.com/1102515903
https://place.map.kakao.com/27148953
https://place.map.kakao.com/495398116
https://place.map.kakao.com/27183515
https://place.map.kakao.com/1927387522
https://place.map.kakao.com/1875054588
https://place.map.kakao.com/214762550
https://place.map.kakao.com/1913617889
https://place.map.kakao.com/1114108866
https://place.map.kakao.com/27527812
https://place.map.kakao.com/738941834
https://place.map.kakao.com/27215452
https://place.map.kakao.com/308342289
https://place.map.kakao.com/8057241
https://place.map.kakao.com/15199519
https://place.map.kakao.com/892348066
https://place.map.kakao.com/1464260459
https://realty.daum.net/home/officetel/danjis/1304/items
https://place.map.kakao.com/13

In [ ]:
columns = ['name', 'score', 'review']
df = pd.DataFrame(columns=columns)

for page_url in page_urls:
    try:
        driver.get(page_url)
        WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '//*[@id="mArticle"]/div[1]/div[1]/div[2]/div/h2')))
        

        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        names = soup.find(name='h2', attrs={"class":"tit_location"})
        
        contents_div = soup.find(name="div", attrs={"class":"evaluation_review"})
        rates = contents_div.find_all(name="em", attrs={"class":"num_rate"})
        reviews = contents_div.find_all(name="p", attrs={"class":"txt_comment"})
    
    
        for rate, review in zip(rates, reviews):
            row = [names.text, rate.text[0], review.find(name="span").text]
            series = pd.Series(row, index=df.columns)
            df = df.append(series, ignore_index=True)

        for button_num in range(2, 6):
            another_reviews = driver.find_element_by_xpath("//a[@data-page='" + str(button_num) + "']")
            another_reviews.click()
            driver.implicitly_wait(5)
            
        
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')

            contents_div = soup.find(name="div", attrs={"class":"evaluation_review"})
            rates = contents_div.find_all(name="em", attrs={"class":"num_rate"})
            reviews = contents_div.find_all(name="p", attrs={"class":"txt_comment"})

            for rate, review in zip(rates, reviews):
                row = [names.text, rate.text[0], review.find(name="span").text]
                series = pd.Series(row, index=df.columns)
                df = df.append(series, ignore_index=True)
    except:
        pass

    
driver.close()

['삼시세끼 셰프반찬', '4', '맛있어요']
['삼시세끼 셰프반찬', '4', '맛있어요']
['삼시세끼 셰프반찬', '4', '맛있어요']
['삼시세끼 셰프반찬', '4', '맛있어요']
['삼시세끼 셰프반찬', '4', '맛있어요']
['삼시세끼 셰프반찬', '4', '맛있어요']
['삼시세끼 셰프반찬', '4', '맛있어요']
['삼시세끼 셰프반찬', '4', '맛있어요']
['삼시세끼 셰프반찬', '4', '맛있어요']
['삼시세끼 셰프반찬', '4', '맛있어요']
['삼시세끼 셰프반찬', '4', '맛있어요']
['삼시세끼 셰프반찬', '4', '맛있어요']
['삼시세끼 셰프반찬', '4', '맛있어요']
['삼시세끼 셰프반찬', '4', '맛있어요']
['삼시세끼 셰프반찬', '4', '맛있어요']
['삼시세끼 셰프반찬', '4', '맛있어요']


In [ ]:
df.to_csv("review_data.csv", index=False)